In [25]:
import pandas as pd
import json

In [26]:
# Read PuG sheet and print information
pug_df = pd.read_excel("./PUG_Prizes.xlsx").astype(str)
pug_df.info()
pug_df["Type"].value_counts()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 610 entries, 0 to 609
Data columns (total 18 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Id                       610 non-null    object
 1   Poster Session           610 non-null    object
 2   Type                     610 non-null    object
 3   Topic                    610 non-null    object
 4   Title                    610 non-null    object
 5   Mantelabstract           610 non-null    object
 6   Abstract                 610 non-null    object
 7   Submitter first name     610 non-null    object
 8   Submitter last name      610 non-null    object
 9   Submitter (affiliation)  610 non-null    object
 10  Submitter (email)        610 non-null    object
 11  Gender                   610 non-null    object
 12  Young scientist          610 non-null    object
 13  IGOR Prize               610 non-null    object
 14  Posterblitz              610 non-null    o

Type
Poster       397
Symposium    213
Name: count, dtype: int64

In [27]:
# Read program sheet and print information
program_df = pd.read_excel("./PUG_2024_Programm_Talks_1504_MD_1804.xlsx").astype(str)
program_df.info()
program_df["Tag"].value_counts()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 182 entries, 0 to 181
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Tag           182 non-null    object
 1   Session       182 non-null    object
 2   SessionStart  182 non-null    object
 3   SessionEnd    182 non-null    object
 4   TalkStart     182 non-null    object
 5   TalkEnd       182 non-null    object
 6   Talk          182 non-null    object
 7   Speaker       182 non-null    object
 8   Chair         182 non-null    object
 9   Inhalt        182 non-null    object
 10  Uni           182 non-null    object
 11  Abstract      182 non-null    object
dtypes: object(12)
memory usage: 17.2+ KB


Tag
Freitag       78
Donnerstag    76
Samstag       25
Mittwoch       3
Name: count, dtype: int64

In [28]:
pug_days_list = program_df["Tag"].unique()
pug_days_dict = dict.fromkeys(pug_days_list)
for key in pug_days_dict.keys():
    s_dict = {"symposia": {}, "poster": {}, "keynote": {}}
    pug_days_dict.update({key: s_dict})

In [29]:
keynote = program_df.loc[program_df["Session"].str.startswith("Keyn")]

for _, k_row in keynote.iterrows():
    pug_days_dict[k_row["Tag"]]["keynote"] = k_row.to_dict()

In [30]:
poster = program_df.loc[program_df["Session"].str.startswith("Post")]

for _, p_row in poster.iterrows():
    pug_days_dict[p_row["Tag"]]["poster"] = p_row.to_dict()	

In [31]:
# Read Symposia from Excel sheet and reset index
symposia = program_df.loc[program_df["Session"].str.startswith("Symp")]
symposia.reset_index(drop=True, inplace=True)
symposia_count = 1

# Iterate over symposia
for s_index, s_row in symposia.iterrows():
    s_id = s_row["Session"]
    s_talk = s_row["Talk"]
    s_title_index = "S" + (str(symposia_count) if symposia_count > 9 else "0" + str(symposia_count))
    s_title = s_title_index + " - " + s_talk
    s_date = s_row["Tag"] + " " + s_row["SessionStart"][:-3] + " - " + s_row["SessionEnd"][:-3]
    s_date_place = s_date + " | Ort TODO"
    s_chairs = "Session chair(s): " + s_row["Chair"]
    s_mantel = s_row["Inhalt"]

    # Add new symposia session if not present
    if s_id not in pug_days_dict[s_row["Tag"]]["symposia"].keys():
        pug_days_dict[s_row["Tag"]]["symposia"][s_id] = {}

    # Add Parameters to symposia session, if not present
    if "Index" not in pug_days_dict[s_row["Tag"]]["symposia"][s_id]:
        symposia_count += 1
        pug_days_dict[s_row["Tag"]]["symposia"][s_id]["Index"] = s_title_index
        pug_days_dict[s_row["Tag"]]["symposia"][s_id]["Talk"] = s_talk
        pug_days_dict[s_row["Tag"]]["symposia"][s_id]["Title"] = s_title
        pug_days_dict[s_row["Tag"]]["symposia"][s_id]["DatePlace"] = s_date_place
        pug_days_dict[s_row["Tag"]]["symposia"][s_id]["Chair"] = s_chairs
        pug_days_dict[s_row["Tag"]]["symposia"][s_id]["Mantel"] = s_mantel
        pug_days_dict[s_row["Tag"]]["symposia"][s_id]["Talks"] = {}
        
    # Add symposium to session
    pug_days_dict[s_row["Tag"]]["symposia"][s_id]["Talks"][s_index] = s_row.to_dict()

In [32]:
# Save the dictionary to a JSON file
with open("PuG_2024_data.json", "w") as file:
    json.dump(pug_days_dict, file)